# Load libraries and data

In [1]:
from pathlib import Path
import re
import numpy as np
import pandas as pd
from collections import Counter
import torch
import torch.nn as nn
import torch.optim as optim
# import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score
import wandb

In [2]:
train_data = pd.read_csv(Path("..", "data", "processed", "train.csv"))
val_data = pd.read_csv(Path("..", "data", "processed", "val.csv"))

# Parameters & wandb

In [3]:
EMBEDDING_DIM = 100
MAX_LEN = 150
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using {DEVICE}")

Using cuda


In [4]:
wandb.login()

# Initialize Weights & Biases
# wandb.init(project="toxic_comment_clf")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: daniele-didino to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# Tokenizer

In [ ]:
# Tokenizer (basic word splitting)
def basic_tokenizer(text: str) -> str:
    text = text.lower()
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text) # Remove special characters
    return text.split()


def build_vocab(texts):
    token_counts = Counter()
    for text in texts:
        cleaned_text = basic_tokenizer(text)
        token_counts.update(cleaned_text)
    vocab = {word: idx + 2 for idx, (word, _) in enumerate(token_counts.most_common())}  # Reserve index 0 for padding, 1 for unknown
    vocab["<PAD>"] = 0
    vocab["<UNK>"] = 1
    return vocab


# Convert texts to numerical sequences
def encode_text(text, vocab, max_len=150):
    tokens = basic_tokenizer(text)
    encoded = [vocab.get(word, vocab["<UNK>"]) for word in tokens[:max_len]]
    return np.pad(encoded, (0, max_len - len(encoded)), constant_values=vocab["<PAD>"])[:max_len]

# Train Model

In [27]:
# Dataset Class
class ToxicDataset(Dataset):
    def __init__(self, texts, labels, vocab, max_len=150):
        texts = [encode_text(text, vocab, max_len) for text in texts]
        self.texts = [torch.tensor(text, dtype=torch.long) for text in texts]
        self.labels = torch.tensor(labels, dtype=torch.float32)

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        return {
            "input_ids": self.texts[idx],
            "labels": self.labels[idx]
        }


# Load Pretrained Embeddings (GloVe)
def load_glove_embeddings(filepath, vocab, embedding_dim=100):
    embeddings = np.random.uniform(-0.25, 0.25, (len(vocab), embedding_dim))
    with open(filepath, 'r', encoding="utf-8") as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype="float32")
            if word in vocab:
                embeddings[vocab[word]] = vector
    return torch.tensor(embeddings, dtype=torch.float32)


# Model: GRU & CNN
class ToxicClassifier(nn.Module):
    def __init__(self, embedding_dim, hidden_dim, kernel_size, dropout, num_classes):
        super(ToxicClassifier, self).__init__()
        self.embedding = nn.Embedding.from_pretrained(embedding_matrix, freeze=False)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.conv = nn.Conv1d(hidden_dim * 2, 32, kernel_size)
        self.pool = nn.AdaptiveMaxPool1d(1)
        self.dropout = nn.Dropout(dropout)
        self.fc = nn.Linear(32, num_classes)

    def forward(self, x):
        x = self.embedding(x)
        x, _ = self.gru(x)
        x = x.permute(0, 2, 1)  # (batch_size, hidden_dim*2, seq_len)
        x = self.conv(x)
        x = self.pool(x).squeeze(2)
        x = self.dropout(x)
        return self.fc(x) # torch.sigmoid(self.fc(x))


# Training function
def model_train(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    model.to(device)

    for epoch in range(epochs):
        model.train()  # set model to training mode
        total_train_loss = 0
        all_preds = []
        all_labels = []

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            outputs = torch.sigmoid(outputs)
            all_preds.append(outputs.cpu().detach().numpy())
            all_labels.append(labels.cpu().numpy())
        
        # Compute Loss
        train_loss = total_train_loss / len(train_loader)

        # Compute AUC_ROC
        all_preds = np.concatenate(all_preds, axis=0).T
        all_labels = np.concatenate(all_labels, axis=0).T
        train_roc_auc = np.mean(
            [roc_auc_score(y_true, y_pred) for y_true, y_pred in zip(all_labels, all_preds)]
        )

        # Validation Step
        model.eval()
        val_preds = []
        val_labels = []
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids)
                
                # Compute validation loss
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()

                val_preds.append(outputs.cpu().numpy())
                val_labels.append(labels.cpu().numpy())
        
        # Compute validation loss
        val_loss = total_val_loss / len(val_loader)

        # Compute AUC_ROC
        val_preds = np.concatenate(val_preds, axis=0).T
        val_labels = np.concatenate(val_labels, axis=0).T
        val_roc_auc = np.mean(
            [roc_auc_score(y_true, y_pred) for y_true, y_pred in zip(val_labels, val_preds)]
        )

        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f} | AUC_ROC: {train_roc_auc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | AUC_ROC: {val_roc_auc:.4f}")

        wandb.log({
            "epoch": epoch + 1,
            "train_loss": train_loss,
            "train_roc_auc": train_roc_auc,
            "val_loss": val_loss,
            "val_roc_auc": val_roc_auc,
        })

    return train_loss, train_roc_auc, val_loss, val_roc_auc


# wandb Sweeps

In [28]:
sweep_config = {
    "method": "bayes", # "random" or "grid" or "bayes"
    "metric": {"name": "val_roc_auc", "goal": "maximize"},
    "parameters": {
        "embed_dim": {"values": [EMBEDDING_DIM]},
        "dropout": {"min": 0.2, "max": 0.5},
        "learning_rate": {"min": 1e-4, "max": 1e-2, "distribution": "log_uniform_values"},
        "batch_size": {"values": [16, 32, 64]},
        # "num_layers": {"min": 1, "max": 3},
        "epochs": {"min": 1, "max": 5},
        "kernel_size": {"values": [3]},
        "hidden_dim": {"values": [128]},
    }
}

# Create the sweep
sweep_id = wandb.sweep(sweep_config, project="toxic_comment_clf")

Create sweep with ID: uvzqse2l
Sweep URL: https://wandb.ai/daniele-didino/toxic_comment_clf/sweeps/uvzqse2l


In [29]:
train_input = train_data.comment_text.to_list()
train_labels = train_data.loc[:, ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

val_input = val_data.comment_text.to_list()
val_labels = val_data.loc[:,  ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

vocab = build_vocab(train_input)

embedding_matrix = load_glove_embeddings("../embedding/glove.6B.100d.txt", vocab, EMBEDDING_DIM)

# Prepare train dataset
train_dataset = ToxicDataset(train_input, train_labels, vocab, MAX_LEN)

# Prepare validation dataset
val_dataset = ToxicDataset(val_input, val_labels, vocab, MAX_LEN)

In [30]:
# Define the training function
def train_sweep():

    EPOCHS = 2
    num_classes = 6  # toxic, severe_toxic, obscene, threat, insult, identity_hate

    with wandb.init() as run:
        config = wandb.config # sample hyperparameters
        
        # Initialize DataLoaders
        train_dataloader = DataLoader(train_dataset, batch_size=config["batch_size"], shuffle=True)
        val_dataloader = DataLoader(val_dataset, batch_size=config["batch_size"], shuffle=True)
    
        # Initialize model
        model = ToxicClassifier(
            config["embed_dim"],
            config["hidden_dim"],
            config["kernel_size"],
            config["dropout"],
            num_classes=num_classes)
        model.to(DEVICE)
    
        # Loss
        criterion = nn.BCEWithLogitsLoss() # Multi-label loss
    
        # Optimizer
        optimizer = optim.Adam(model.parameters(), lr=config["learning_rate"])
    
        # Training loop
        _ = model_train(model, train_dataloader, val_dataloader, criterion, optimizer, EPOCHS, DEVICE)
        # _ = model_train(model, train_dataloader, val_dataloader, criterion, optimizer, config["epochs"], DEVICE)

In [31]:
# Launch the sweep
wandb.agent(sweep_id, function=train_sweep, count=2)

wandb: Agent Starting Run: b3jtshen with config:
wandb: 	batch_size: 32
wandb: 	dropout: 0.31899612627316104
wandb: 	embed_dim: 100
wandb: 	epochs: 5
wandb: 	hidden_dim: 128
wandb: 	kernel_size: 3
wandb: 	learning_rate: 0.004154940765088244


Epoch 1/2
Train Loss: 0.0593 | AUC_ROC: 0.9570
Val Loss: 0.0505 | AUC_ROC: 0.9835
Epoch 2/2
Train Loss: 0.0401 | AUC_ROC: 0.9847
Val Loss: 0.0496 | AUC_ROC: 0.9818


epoch,▁█
train_loss,█▁
train_roc_auc,▁█
val_loss,█▁
val_roc_auc,█▁
epoch,2
train_loss,0.04008
train_roc_auc,0.98468
val_loss,0.04964
val_roc_auc,0.9818


wandb: Agent Starting Run: mrqwxnvc with config:
wandb: 	batch_size: 16
wandb: 	dropout: 0.23839793533083883
wandb: 	embed_dim: 100
wandb: 	epochs: 5
wandb: 	hidden_dim: 128
wandb: 	kernel_size: 3
wandb: 	learning_rate: 0.004783420078404673


Epoch 1/2
Train Loss: 0.0627 | AUC_ROC: 0.9503
Val Loss: 0.0534 | AUC_ROC: 0.9777
Epoch 2/2
Train Loss: 0.0490 | AUC_ROC: 0.9746
Val Loss: 0.0550 | AUC_ROC: 0.9766


epoch,▁█
train_loss,█▁
train_roc_auc,▁█
val_loss,▁█
val_roc_auc,█▁
epoch,2
train_loss,0.049
train_roc_auc,0.97457
val_loss,0.05496
val_roc_auc,0.97658


# CANC

In [32]:
BATCH_SIZE = 32
EPOCHS = 2
HIDDEN_DIM = 128
KERNEL_SIZE = 3
DROPOUT = 0.5
LEARNING_RATE = 1e-3

In [ ]:
train_input = train_data.comment_text.to_list()
train_labels = train_data.loc[:, ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

val_input = val_data.comment_text.to_list()
val_labels = val_data.loc[:,  ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]].values.tolist()

# Prepare train dataset
train_dataset = ToxicDataset(train_input, train_labels, vocab, MAX_LEN)
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Prepare validation dataset
val_dataset = ToxicDataset(val_input, val_labels, vocab, MAX_LEN)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)

In [34]:
# Training function
def model_train2(model, train_loader, val_loader, criterion, optimizer, epochs, device):
    model.to(device)

    for epoch in range(epochs):
        model.train()  # set model to training mode
        total_train_loss = 0
        all_preds = []
        all_labels = []

        for batch in train_loader:
            input_ids = batch['input_ids'].to(device)
            labels = batch['labels'].to(device)

            optimizer.zero_grad()
            outputs = model(input_ids)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_train_loss += loss.item()
            outputs = torch.sigmoid(outputs)
            all_preds.append(outputs.cpu().detach().numpy())
            all_labels.append(labels.cpu().numpy())
        
        # Compute Loss
        train_loss = total_train_loss / len(train_loader)

        # Compute AUC_ROC
        all_preds = np.concatenate(all_preds, axis=0).T
        all_labels = np.concatenate(all_labels, axis=0).T
        train_roc_auc = np.mean(
            [roc_auc_score(y_true, y_pred) for y_true, y_pred in zip(all_labels, all_preds)]
        )

        # Validation Step
        model.eval()
        val_preds = []
        val_labels = []
        total_val_loss = 0
        with torch.no_grad():
            for batch in val_loader:
                input_ids = batch['input_ids'].to(device)
                labels = batch['labels'].to(device)
                outputs = model(input_ids)
                
                # Compute validation loss
                loss = criterion(outputs, labels)
                total_val_loss += loss.item()

                val_preds.append(outputs.cpu().numpy())
                val_labels.append(labels.cpu().numpy())
        
        # Compute validation loss
        val_loss = total_val_loss / len(val_loader)

        # Compute AUC_ROC
        val_preds = np.concatenate(val_preds, axis=0).T
        val_labels = np.concatenate(val_labels, axis=0).T
        val_roc_auc = np.mean(
            [roc_auc_score(y_true, y_pred) for y_true, y_pred in zip(val_labels, val_preds)]
        )

        print(f"Epoch {epoch + 1}/{epochs}")
        print(f"Train Loss: {train_loss:.4f} | AUC_ROC: {train_roc_auc:.4f}")
        print(f"Val Loss: {val_loss:.4f} | AUC_ROC: {val_roc_auc:.4f}")

    return train_loss, train_roc_auc, val_loss, val_roc_auc


In [35]:
BATCH_SIZE = 32
EPOCHS = 2
HIDDEN_DIM = 128
KERNEL_SIZE = 3
DROPOUT = 0.5
LEARNING_RATE = 1e-3

In [36]:
model_2 = ToxicClassifier(
    EMBEDDING_DIM,
    HIDDEN_DIM,
    KERNEL_SIZE,
    DROPOUT,
    num_classes=6).to(DEVICE)

criterion_2 = nn.BCEWithLogitsLoss()  # Multi-label loss
optimizer_2 = optim.Adam(model_2.parameters(), lr=LEARNING_RATE)

history = model_train2(
    model_2,
    train_dataloader,
    val_dataloader,
    criterion_2, 
    optimizer_2,
    EPOCHS,
    DEVICE)

Epoch 1/2
Train Loss: 0.0629 | AUC_ROC: 0.9519
Val Loss: 0.0479 | AUC_ROC: 0.9832
Epoch 2/2
Train Loss: 0.0433 | AUC_ROC: 0.9821
Val Loss: 0.0449 | AUC_ROC: 0.9855
